In [1]:
import torch
import functions
import modules
import math
from optimizer import SGD

from torch import optim
from torch import Tensor
from torch import nn

# torch.set_grad_enabled(False)

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

## Test.py

## Generate data

In [2]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [3]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [4]:
train_input, train_target = generate_disc_set(3)

In [5]:
print(train_input)
print(train_target)

tensor([[0.9628, 0.1617],
        [0.8909, 0.1638],
        [0.7142, 0.8702]])
tensor([1, 1, 1])


In [6]:
train_target = convert_to_one_hot_labels(train_input, train_target)

In [7]:
print(train_input)
print(train_target)

tensor([[0.9628, 0.1617],
        [0.8909, 0.1638],
        [0.7142, 0.8702]])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.]])


In [8]:
train_input, train_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)

In [9]:
print(train_input.shape)
print(train_target.shape)

torch.Size([1000, 2])
torch.Size([1000, 2])


In [10]:
arch1 = modules.Sequential(modules.Linear(2, 2))

In [11]:
for name, m in arch1.__dict__.items():
    print(name)
    print(m)

Linear0


In [12]:
nb_epochs = 10
lr = 1e-1
optimizer = SGD(arch1.param(), lr=lr)
criterion = modules.MSELoss()


for e in range(nb_epochs):
    output = arch1.forward(train_input)
    loss = criterion.forward(output, train_target)
    gradwrtoutput = criterion.backward()
    grad = arch1.backward(gradwrtoutput)
    print("Loss: ", loss)
    optimizer.zero_grad()
    optimizer.step()

Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)
Loss:  tensor(999.9985)


In [13]:
model = nn.Sequential(
            nn.Linear(2, 2),
#             nn.ReLU(),
#             nn.Linear(128, 2)
        )
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
nb_epochs = 10

for e in range(nb_epochs):
    output = model(train_input)
    loss = criterion(output, train_target.float())
    print(loss)
    model.zero_grad()
    loss.backward()
    optimizer.step()

tensor(0.5714, grad_fn=<MseLossBackward>)
tensor(0.4423, grad_fn=<MseLossBackward>)
tensor(0.3496, grad_fn=<MseLossBackward>)
tensor(0.2830, grad_fn=<MseLossBackward>)
tensor(0.2351, grad_fn=<MseLossBackward>)
tensor(0.2007, grad_fn=<MseLossBackward>)
tensor(0.1758, grad_fn=<MseLossBackward>)
tensor(0.1579, grad_fn=<MseLossBackward>)
tensor(0.1449, grad_fn=<MseLossBackward>)
tensor(0.1354, grad_fn=<MseLossBackward>)


## Mini-Project Architecture

In [14]:
seq = modules.Sequential(modules.Linear(2, 25),
                         modules.ReLU(),
                         modules.Linear(25, 25),
                         modules.ReLU(),
                         modules.Linear(25, 25),
                         modules.ReLU(),
                         modules.Linear(25, 25),
                         modules.ReLU(),
                         modules.Linear(25, 2))

In [15]:
for name, m in seq.__dict__.items():
    print(name)
    print(m)

Linear0
ReLU1
Linear2
ReLU3
Linear4
ReLU5
Linear6
ReLU7
Linear8


In [16]:
nb_epochs = 10
lr = 1e-1
optimizer = SGD(seq.param(), lr=lr)
criterion = modules.MSELoss()


for e in range(nb_epochs):
    output = seq.forward(train_input)
    loss = criterion.forward(output, train_target)
    print("Loss: ", loss)
    gradwrtoutput = criterion.backward()
    print(gradwrtoutput.shape)
    grad = seq.backward(gradwrtoutput)
    optimizer.zero_grad()
    optimizer.step()

Loss:  tensor(999.9998)
torch.Size([1000, 2])


RuntimeError: size mismatch, m1: [1000 x 2], m2: [25 x 2] at /Users/soumith/mc3build/conda-bld/pytorch_1549593514549/work/aten/src/TH/generic/THTensorMath.cpp:940